In [1]:
def getRules(rule):
    ruleId = rule.split(':')[0]
    temp = rule.split(':')[1]
    if('"' in temp):
        return ruleId, rule.split(':')[1].replace('"','').strip()
    else:
        return ruleId, [rule.strip().split(' ') for rule in rule.split(':')[1].split('|')]    

In [2]:
fileInput = open('data.txt', mode='r')
dataInput = [group.split('\n') for group in fileInput.read().split('\n\n')]
rulesInput = {getRules(rule)[0]:getRules(rule)[1] for rule in dataInput[0]}
fileInput.close()

In [3]:
from functools import reduce
from itertools import product

def mergePossibilities(poss1,poss2):
    return [p1+p2 for p1, p2 in product(poss1,poss2)]

def specialConcat(listOfPoss):
    return reduce(lambda a,b : mergePossibilities(a,b), listOfPoss)

def getFinalRule(ruleId, rules):
    rule = rules[ruleId]
    if not(isinstance(rule,list)): return [rule]
    else:
        rule = [specialConcat([getFinalRule(i,rules) for i in r]) for r in rule]
        return [p for pos in rule for p in pos]

In [4]:
#r8 = r42 ou r42 + r8
#r11 = r42 + r31 ou r42 + r11 + r31
# r0 = r8 + r11
rules42 = getFinalRule('42', rulesInput)
rules31 = getFinalRule('31', rulesInput)
rules11_1 = mergePossibilities(rules42,rules31)

def specialAnalyseForR11(text):
    if(text in rules11_1): return True
    else:
        nbChar42, nbChar31 = len(rules42[0]), len(rules31[0])
        if(text[0:nbChar42] in rules42 and text[-nbChar31:] in rules31):
            return specialAnalyseForR11(text[nbChar42:(len(text)-nbChar31)])
        return False

def analyseText(text):
    nbChar42 = len(rules42[0])
    if(len(text)%nbChar42 != 0): return False
    i=0
    while(i+nbChar42<len(text) and text[i:i+nbChar42] in rules42):
        i+=nbChar42
    return any([specialAnalyseForR11(text[i-j*nbChar42::]) for j in range(i//nbChar42)])

In [5]:
listRules = getFinalRule('0', rulesInput)
print(f'Part 1 : {len([text for text in dataInput[1] if text in listRules])}')
print(f'Part 2 : {len([text for text in dataInput[1] if analyseText(text)])}')

Part 1 : 139
Part 2 : 289
